In [1]:
from datasets import load_dataset

/Users/charlesoneill/miniconda3/envs/jsalt/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
path = "charlieoneill/jsalt-astroph-dataset"
dataset = load_dataset(path)

In [3]:
# Get the entries where arxiv_id is 1607.02388
entries = dataset["train"].filter(lambda x: x["arxiv_id"] == "1607.02388")
print(entries)

Filter: 100%|██████████| 271544/271544 [00:06<00:00, 44600.38 examples/s]

Dataset({
    features: ['subfolder', 'filename', 'abstract', 'introduction', 'conclusions', 'year', 'month', 'arxiv_id'],
    num_rows: 1
})


In [5]:
entries['abstract']

[' ']

In [7]:
entries['conclusions']

['']

In [8]:
entries['introduction']

["The initial mass of a star on the zero-age main sequence largely determines its fate. Stars born with masses exceeding $\\sim 10\\,M_\\odot$ end their lives in a core-collapse event, e.g. a supernova (SN) explosion, and leave a neutron star or a black hole as remnant \\citep{Heger2003}. Such massive stars are luminous, with bolometric luminosities exceeding $L_{\\rm bol}\\msim 10^4\\,L_\\odot$. On the main sequence, massive stars have spectral types earlier than B2V. These bright stars live very fast, the most massive of them die within just $\\sim 10$\\,Myr. Albeit we see many massive stars by naked eye in the night sky (e.g.\\ the Orion Belt consists of massive stars), these stars are actually very rare and constitute only $\\sim 0.4$\\%\\ of all stars in our Milky Way.  Despite their small number, massive stars have enormous impact on the galactic ecology. Their strong ionizing radiation and stellar winds, as well as their final demise in  SN explosions, largely determine the phys

In [16]:
import feedparser
import urllib, urllib.request

def get_abstract(arxiv_id):

    query = arxiv_id
    url = 'http://export.arxiv.org/api/query?search_query='+query+'&start=0&max_results=1'
    data = urllib.request.urlopen(url)
    # print(data.read().decode('utf-8'))
    data = data.read().decode('utf-8')
    feed = feedparser.parse(data)
    return feed['entries'][0]['summary']

abstract = 'physics/9804027'
get_abstract(abstract)

'It is shown that the plasma, generated during an impact of a meteoroid with\nan artificial satellite, can produce electromagnetic radiation below the\nmicrowave frequency range. This interference is shown to exceed local noise\nsources and might disturb regular satellite operations.'

In [14]:
from datasets import load_dataset

ds = load_dataset("JSALT2024-Astro-LLMs/astro_paper_corpus")

Trying to resume download...
Trying to resume download...
Generating train split: 100%|██████████| 271544/271544 [00:04<00:00, 61724.03 examples/s]
